# Jyupyter ノートブック上での前処理、学習、推論

このノートブックではデータの前処理、学習、推論を Jupyter ノートブック上で行います。 モデルは [`scikit-learn` ](https://scikit-learn.org/) のランダムフォレストを活用します。特徴量作成やアルゴリズムの選択、ハイパーパラメータの探索など、精度向上の余地などはありますが、まずはパイプラインのベースとします。 Code にあった [Titanic: random forest using sklearn and pandas](https://www.kaggle.com/edeastwood/titanic-random-forest-using-sklearn-and-pandas) を参考にさせて頂きました。

In [1]:
import joblib
import re

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

データを読み込みます。

In [2]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## データの前処理

特徴量作成など、データの前処理を行います。特徴量作成や欠損値の丁寧な取り扱いなど、精度向上のために多くの工夫ができるかと思いますが、機械学習パイプライン構築のためのベースとして行っています。

*特徴量を作成する際のアプローチ*
- 氏名に含まれるタイトルをカテゴリに変換
- 連れている子供や両親の数、同行者中の子供と大人の比率などを算出
- 同行者がいない子供のみかどうか
といった、家族構成やその中での立ち位置に注目したもの。詳細は、[Titanic: random forest using sklearn and pandas](https://www.kaggle.com/edeastwood/titanic-random-forest-using-sklearn-and-pandas) をご確認下さい。

In [5]:
train['data_type'] = "train"
test['data_type'] = "test"
test['Survived'] = "test"

data = pd.concat([train, test], axis=0)
data = data.drop(['Cabin'], axis=1)
data = data.dropna(how="any")

In [6]:
def deriveTitles(name: str) -> str:
    """ name に含まれるタイトルをそれぞれのカテゴリに変換
    Args:
        name (str): 登場者の名前
    Returns:
        名前から抽出されたタイトルに従って変換されたカテゴリ (str)
    """
    title = re.search('(?:\S )(?P<title>\w*)', name).group('title')
    
    if title == "Mr":             return "adult"
    elif title == "Don":          return "gentry"
    elif title == "Dona":         return "gentry"
    elif title == "Miss":         return "miss"
    elif title == "Col":          return "military"
    elif title == "Rev":          return "other"
    elif title == "Lady":         return "gentry"
    elif title == "Master":       return "child"
    elif title == "Mme":          return "adult"
    elif title == "Captain":      return "military"
    elif title == "Dr":           return "other"
    elif title == "Mrs":          return "adult"
    elif title == "Sir":          return "gentry"
    elif title == "Jonkheer":     return "gentry"
    elif title == "Mlle":         return "miss"
    elif title == "Major":        return "military"
    elif title == "Ms":           return "miss"
    elif title == "the Countess": return "gentry"
    else:                         return "other"
    
data["title"] = data.Name.apply(deriveTitles)
# and encode these new titles for later
le = preprocessing.LabelEncoder()
titles = ['adult', 'gentry', 'miss', 'military', 'other', 'child']
le.fit(titles)
data['encodedTitle'] = le.transform(data['title']).astype('int')

In [7]:
data = data.assign(SibSpGroup1=train['SibSp'] < 2)
data = data.assign(SibSpGroup2=train['SibSp'].between(2, 3, inclusive=True))
data = data.assign(SibSpGroup3=train['SibSp'] > 2)

In [8]:
data = data.assign(ParChGT2=train['Parch'] > 2)
data = data.assign(familySize=data['Parch'] + data['SibSp'])

In [9]:
def deriveChildren(age: int, parch: int) -> int:
    """ 連れている両親の数だけを抽出。18歳以上は両親がいないと仮定。
    Args:
        age (int): 搭乗者の年齢
        parch (int): 両親、子供の数
    Returns:
        両親の数 (int)
    """
    if(age < 18):
        return parch
    else:
        return 0
    
data = data.assign(children=data.apply(lambda row: deriveChildren(row['Age'], row['Parch']), axis = 1))

In [10]:
def deriveParents(age: int, parch: int) -> int:
    """ 連れている子供の数だけを抽出。18歳未満は子供がいないと仮定。
    Args:
        age (int): 搭乗者の年齢
        parch (int): 両親、子供の数
    Returns:
        子供の数 (int)
    """
    if(age > 17):
        return parch
    else:
        return 0
    
data['parents'] = data.apply(lambda row: deriveParents(row['Age'], row['Parch']), axis = 1)

In [11]:
def deriveResponsibleFor(children: int, SibSp: int) -> float:
    """ 自分を含めた同行者における18歳以上のの大人と責任を持つべき子供の割合を算出。
    Args:
        children (int): 子供の数
        SibSp (int): 兄弟、配偶者の数
    Returns:
        責任 (float)
    """
    if(children > 0):
        return children / (SibSp + 1)  
    else:
        return 0

data['responsibleFor'] = data.apply(lambda row: deriveResponsibleFor(row['children'], row['SibSp']), axis = 1)

In [12]:
def deriveAccompaniedBy(parents, SibSp):
    """ 親が1以上いる場合に、自分を含めた同行者における、両親の割合を算出。
    Args:
        parents (int): 親の数
        SibSp (int): 兄弟、配偶者の数
    Returns:
        親に対する子供の割合 (float)
    """
    if(parents > 0): return parents / (SibSp + 1)
    else: return 0

data['accompaniedBy'] = data.apply(lambda row: deriveAccompaniedBy(row['parents'], row['SibSp']), axis = 1)

In [13]:
def unaccompaniedChild(age: int, parch: int) -> bool:
    """ 同行者がいない16歳以下の搭乗者かどうかを判定。
    Args:
        age (int): 搭乗者の年齢
        parch (int): 同行者の数
    Returns:
        同行者がいない場合 True, いる場合 False (bool)
    """
    if((age < 16) & (parch == 0)):
        return 1
    else:
        return 0
    
data['unaccompaniedChild'] = data.apply(lambda row: unaccompaniedChild(row['Age'], row['Parch']), axis = 1)

In [14]:
data = data.astype({"Pclass": int, "Age": int, "SibSp": int, "Parch": int})

# label encode string features
categorical_names = {}
categorical_features = ['Embarked', 'Sex', 'SibSpGroup1', 'SibSpGroup2', 'SibSpGroup3', 'ParChGT2']

for feature in categorical_features:
    le = preprocessing.LabelEncoder()
    le.fit(data[feature])
    data[feature] = le.transform(data[feature])
    categorical_names[feature] = le.classes_


data['class'] = data['Pclass'].astype(int, copy=False)
data = data.drop(['Pclass'], axis=1)
data = data.drop(['Name', 'Parch', 'SibSp', 'title', 'Ticket', 'PassengerId'], axis=1)

In [15]:
train = data[data['data_type'] == 'train']
test = data[data['data_type'] == 'test']
X_train, y_train = train.drop(['Survived', 'data_type'], axis=1), train['Survived']
y_train = y_train.astype({"Survived": int})

test = test.drop(['Survived', 'data_type'], axis=1)

In [17]:
!mkdir ./data/preprocessed

X_train.to_csv('./data/preprocessed/X_train.csv', header=True, index=False)
y_train.to_csv('./data/preprocessed/y_train.csv', header=True, index=False)
test.to_csv('./data/preprocessed/test.csv', header=True, index=False)

## 機械学習モデルの学習

今回は `scikit-learn` のランダムフォレストを学習させます。 ハイパーパラメーターの探索は、 `criterion` と `max_depth` のみをグリッドサーチで実施しました。グリッドサーチでのハイパラの評価は、評価指標として `f1` を使用し、交差検証を 3-fold で実施ししています。単純に `f1` スコアが良かったものをベストモデルとしました。

In [18]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

param_grid =  {
    "criterion": ["gini", "entropy"],
    "max_depth":[i for i in range(1, 5)]
              }

stratifiedkfold = StratifiedKFold(n_splits=3)
grid_search = GridSearchCV(rf, param_grid, cv=stratifiedkfold, refit=True, scoring='f1')
grid_search.fit(X_train, y_train)

print("score: {}".format(grid_search.best_score_))

score: 0.7466106073522955


## 学習したランダムフォレストモデルによる推論

学習したモデルにて推論しています。

In [19]:
prediction = grid_search.predict(test)
pd.Series(prediction, name='Survived').to_csv('./data/prediction.csv', header=True, index=False)

Jupyter ノートブック上での前処理、学習、推論を実施しました。